In [1]:
# set the python path to the parent directory
# this allows us to import from the psychonaut package
import sys
from pathlib import Path
from pprint import pprint

sys.path.append(str(Path.cwd().parent))

In [2]:
retain = []
def err_callback(i, msg):
    retain.append((i, msg))

In [12]:
from psychonaut.firehose.serde import length_prefixed_iter, read_msg_pair
from psychonaut.firehose.simple_reader import iter_events
from collections import defaultdict
import random

sample_segment = Path("../../stream_length_prefixed/stream.1682780863.length-prefixed")
msgs = length_prefixed_iter(sample_segment)
op_codes = defaultdict(int)
sample = []
for i, event in enumerate(iter_events(msgs, err_callback)):
    op_codes[(event[0]['t'], event[0]['op'])] += 1
    if random.random() < 0.001:
        sample.append(event)

op_codes

defaultdict(int, {('#commit', 1): 29005})

In [13]:
len(retain)

2

In [14]:
from psychonaut.firehose.car import read_car


error_target = retain[-1]
read_car(error_target[1])

OSError: Failed to read full buffer

In [18]:
len(error_target[1])

117

In [15]:
jjsample[0]

({'t': '#commit', 'op': 1},
 {'ops': [{'cid': CID('base58btc', 1, 'dag-cbor', '1220427c6e8599870c68566ef338994b153ce5235ff6f7c6384f75e15bd29a6f46a2'),
    'path': 'app.bsky.feed.like/3jujfu2b2lt2g',
    'action': 'create'}],
  'seq': 8982051,
  'prev': CID('base58btc', 1, 'dag-cbor', '1220ea8071132b4749c9370cc985329f9bd13cf74d499b570ce07bb308f5e4e55f8f'),
  'repo': 'did:plc:tnvlrgeen4b5y4uzmvnrljb5',
  'time': '2023-04-29T15:07:50.397Z',
  'blobs': [],
  'blocks': b':\xa2eroots\x81\xd8*X%\x00\x01q\x12 \xecd\xf0\xc1\x84*\xf69\xfdL\xe2p 7\x08U(\xc9\x04\x16\x9c\xd9>f\xf0\x9f\xc4bj\xb0\x7f\xe1gversion\x01\xf8\x01\x01q\x12 B|n\x85\x99\x87\x0chVn\xf38\x99K\x15<\xe5#_\xf6\xf7\xc68Ou\xe1[\xd2\x9aoF\xa2\xa3e$typerapp.bsky.feed.likegsubject\xa2ccidx;bafyreigwjojfw3fthyrzxagnjm5wm2jn2f2ozcmwh3r2k65h4glg7fvqwacurixFat://did:plc:oky5czdrnfjpqslsw2a5iclo/app.bsky.feed.post/3juhxymund22ricreatedAtx\x182023-04-29T15:07:50.211Z\xf4\x04\x01q\x12 \xfa\x9f\x19\x96\xd8~\x94\x12\xd8\xb6\xa7^D\xa2\x06\xa3 \x

In [40]:
from typing import Optional
from pydantic import BaseModel
from datetime import datetime
from dateutil.parser import parse as parse_date


class Like(BaseModel):
    cid: str
    path: str
    action: str
    repo: str
    created_at: datetime

class Follow(BaseModel):
    cid: str
    path: str
    action: str
    repo: str
    subject: str
    created_at: datetime

class Post(BaseModel):
    cid: Optional[str]
    path: str
    action: str
    repo: str
    created_at: datetime
    text: str


likes = []
follows = []

def dumb_iterator(items):
    for kind, obj, roots, blocks in items:
        if kind.get('t') != '#commit' or kind.get('op') != 1:
            continue

        ops = obj.get('ops')
        if not ops:
            continue

        if 'app.bsky.feed.like' in ops[0]['path']:
            like = Like(
                cid=str(ops[0]['cid']),
                path=ops[0]['path'],
                action=ops[0]['action'],
                repo=obj['repo'],
                created_at=parse_date(obj['time'])
            )
            likes.append(like)
        elif 'app.bsky.graph.follow' in ops[0]['path']:
            follow = Follow(
                cid=str(ops[0]['cid']),
                path=ops[0]['path'],
                action=ops[0]['action'],
                repo=obj['repo'],
                subject=blocks[0].decoded['subject'],
                created_at=parse_date(obj['time'])
            )
            follows.append(follow)
        else:
            print(blocks[0].decoded)
            pass

        yield kind, obj, roots, blocks


for lmao in dumb_iterator(sample):
    pass

len(likes), len(follows)

{'e': [{'k': b'app.bsky.feed.post/3ju4cmyhckr2g', 'p': 0, 't': None, 'v': CID('base58btc', 1, 'dag-cbor', '1220f18f8846aada47013fcae5bb29bfa6a5b8f262278a54f594ef49f0c5d33df1e2')}, {'k': b'zwligg2u', 'p': 24, 't': CID('base58btc', 1, 'dag-cbor', '122077325a338d27be114673e763e6166ab9c32e4e836d498087c4de8ff416f1a504'), 'v': CID('base58btc', 1, 'dag-cbor', '12206371eeea8021513029302c83568b0b5aeb11b1d810e799ea84a1e0fe28e71bbb')}, {'k': b'7gczavnc2x', 'p': 22, 't': CID('base58btc', 1, 'dag-cbor', '1220692e80c7cf2d64725fac787a5861024ea28081f040420354794e378e9516ae0f'), 'v': CID('base58btc', 1, 'dag-cbor', '1220373a47ede40cc4f61cba9613d213802af2ec61d2bc2c5342c81e149bf21939d9')}, {'k': b'ed4ymndj2o', 'p': 22, 't': CID('base58btc', 1, 'dag-cbor', '1220dd37f7eb33c33b5d07e4131c12153cd885cf00339df2ffe7a279c278af4ec36f'), 'v': CID('base58btc', 1, 'dag-cbor', '12207f56ac5e65d9d11c8e30abeb1ffa01745f0ba66390baa95e1571b85785cdff0b')}], 'l': CID('base58btc', 1, 'dag-cbor', '122096a9ecb892c3b8b4f5ef9affd7

(16, 16)

In [24]:
likes

[Like(cid='zdpuAptySNpcJxsn1KuELBAMCo9wfeYPLRU9RuTFfMANCbzUZ', path='app.bsky.feed.like/3jujfu2b2lt2g', action='create', repo='did:plc:tnvlrgeen4b5y4uzmvnrljb5', created_at=datetime.datetime(2023, 4, 29, 15, 7, 50, 397000, tzinfo=tzutc())),
 Like(cid='zdpuAt72tLw3VKjwq9KaHowV9pdHDtEJouTYHx7kT7dDKnp9U', path='app.bsky.feed.like/3jujfwqusae23', action='create', repo='did:plc:buofnbcavecxm3kr6x5npusi', created_at=datetime.datetime(2023, 4, 29, 15, 9, 21, 269000, tzinfo=tzutc())),
 Like(cid='zdpuAkh5MyyH9ZLvn3buYavvK4J2BjyKvGkMj7WppHvJ694Pa', path='app.bsky.feed.like/3jujg6xc7nd2o', action='create', repo='did:plc:vazqcdlccpdlpqn5mki5dpy2', created_at=datetime.datetime(2023, 4, 29, 15, 13, 56, 379000, tzinfo=tzutc())),
 Like(cid='zdpuAtTmtQ42CwPQfBznDEk8hDdP7mKPr4qBsyVCqVZkaCm6s', path='app.bsky.feed.like/3jujgbhq7se2j', action='create', repo='did:plc:2toeszg7aqbwvfrsim3x6gh7', created_at=datetime.datetime(2023, 4, 29, 15, 15, 20, 744000, tzinfo=tzutc())),
 Like(cid='zdpuAkvkD7Tri7x4ZwoSYD1